## This script produced 20 shuffled FLARE bed file (shuffle within the region)

In [1]:
import os
cwd = os.getcwd()
main_dir = cwd
print(main_dir)

/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks


In [2]:
import pandas as pd
import pybedtools
from glob import glob


### Load peak files: cleaned confident peaks

In [7]:
peak_filepaths = sorted(glob(main_dir + '/confident_clusters/*_cleaned_confident_clusters.bed'))

print(len(peak_filepaths))

file_id_to_df = {}

for peak_filepath in peak_filepaths:    
    print(peak_filepath)
    file_id = peak_filepath.split('/')[-1].split('_cleaned_confident')[0]
    print('\t', file_id)
    file_id_to_df[file_id] = pd.read_csv(peak_filepath, sep='\t', header = None, names = ['chrom', 'start', 'end', 'depth', 'fractions', 'strand', 'region'])

print(len(file_id_to_df))

3
/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/confident_clusters/Enzyme_Only_PFA_cleaned_confident_clusters.bed
	 Enzyme_Only_PFA
/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/confident_clusters/RBFOX2_INSCRIBE_PFA_cleaned_confident_clusters.bed
	 RBFOX2_INSCRIBE_PFA
/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/confident_clusters/TDP43_INSCRIBE_PFA_cleaned_confident_clusters.bed
	 TDP43_INSCRIBE_PFA
3


### Make bedtools from peak files

In [8]:
file_id_to_bedtool = {}
for file_id, df in file_id_to_df.items():
    print(file_id)
    bedtool = pybedtools.BedTool.from_dataframe(df[['chrom', 'start', 'end', 'strand']])
    print('\t', len(bedtool))
    file_id_to_bedtool[file_id] = bedtool

Enzyme_Only_PFA
	 1678
RBFOX2_INSCRIBE_PFA
	 2970
TDP43_INSCRIBE_PFA
	 2540


### Load regions in which to shuffle STAMP peaks

In [9]:
# Load regions
region_map_bedtool = pybedtools.BedTool('/tscc/projects/ps-yeolab3/ekofman/ReferenceData/peakcalling_regions/hg38/hg38_cellranger_region_map.bed')
len(region_map_bedtool)

def add_index(r):
    return '{}:{}-{}({})'.format(r.chrom, r.start, r.end, r.strand)

names = 'chrom   start   end     edit_fraction   strand  target_bases    edited_bases    num_edited_reads        total_reads_in_region   fraction_reads_edited   mean_depth      num_substrate_bases     subregion_coverage      subregion_conversions   region_coverage region_conversions      gene_coverage   gene_conversions        score'.split()


### format conversion

In [10]:
conversion = {
    'chr1' : '1', 
    'chr10': '10', 
    'chr11': '11', 
    'chr12': '12', 
    'chr13': '13', 
    'chr14': '14', 
    'chr15': '15',
    'chr16': '16',
    'chr17': '17', 
    'chr18': '18', 
    'chr19': '19', 
    'chr2' : '2', 
    'chr20': '20', 
    'chr21': '21',
    'chr22': '22', 
    'chr3' : '3', 
    'chr4' : '4', 
    'chr5' : '5',
    'chr6' : '6', 
    'chr7' : '7', 
    'chr8' : '8', 
    'chr9' : '9',
    'chrM' : 'MT',
    'chrX' : 'X', 
    'chrY' : 'Y'
}

conversion = {v:k for k,v in conversion.items()}


def change(c):
    return conversion.get(c)

region_map_df = region_map_bedtool.to_dataframe()
region_map_df['chrom'] = region_map_df.chrom.astype(str).apply(change)
region_map_bedtool = pybedtools.BedTool.from_dataframe(region_map_df)

/tscc/nfs/home/q2liang/miniconda3/envs/py3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Permute the STAMP peaks within their respective region

In [11]:
file_id_to_intersected = {}

for file_id, bedtool in file_id_to_bedtool.items():
    print(file_id)
    intersected = region_map_bedtool.intersect(bedtool, wa=True, wb=True).to_dataframe(names=['chrom_region', 'start_region', 'end_region', 'symbol_region', 'label_region', 'strand_region'] +names )

    intersected.index = intersected.apply(add_index, axis=1)
    intersected = intersected[(intersected.start >= intersected.start_region) & (intersected.end <= intersected.end_region)]
    intersected['peak_size'] = intersected['end'] - intersected['start']
    intersected = intersected[~intersected.index.duplicated(keep='first')]
    print('\t', len(intersected), len(bedtool))
    file_id_to_intersected[file_id] = intersected
    

Enzyme_Only_PFA


***** WARNING: File /tmp/pybedtools.gjhvq2pu.tmp has inconsistent naming convention for record:
	56140	58376	exon	BX004987.1	-

***** WARNING: File /tmp/pybedtools.gjhvq2pu.tmp has inconsistent naming convention for record:
	56140	58376	exon	BX004987.1	-



	 1492 1678
RBFOX2_INSCRIBE_PFA


***** WARNING: File /tmp/pybedtools.gjhvq2pu.tmp has inconsistent naming convention for record:
	56140	58376	exon	BX004987.1	-

***** WARNING: File /tmp/pybedtools.gjhvq2pu.tmp has inconsistent naming convention for record:
	56140	58376	exon	BX004987.1	-



	 2709 2970
TDP43_INSCRIBE_PFA
	 2267 2540


***** WARNING: File /tmp/pybedtools.gjhvq2pu.tmp has inconsistent naming convention for record:
	56140	58376	exon	BX004987.1	-

***** WARNING: File /tmp/pybedtools.gjhvq2pu.tmp has inconsistent naming convention for record:
	56140	58376	exon	BX004987.1	-



In [12]:
import random

NUM_RAND_REGIONS = 20

def get_randomized_region(chrom, start, end, strand, peak_size):
    random_regions = []
    for n in range(NUM_RAND_REGIONS):
        random_start = random.randint(start, end-peak_size)
        random_end = random_start + peak_size
        
        random_region = '{}:{}-{}({})'.format(chrom, random_start, random_end, strand)
        random_regions.append(random_region)
    
    return random_regions
    
def get_random_regions(chrom, start, end, peak_start, peak_end, strand):
    peak_size = peak_end - peak_start
    return get_randomized_region(chrom, start, end, strand, peak_size)
    

In [13]:
file_id_to_intersected.keys()

dict_keys(['Enzyme_Only_PFA', 'RBFOX2_INSCRIBE_PFA', 'TDP43_INSCRIBE_PFA'])

In [14]:
file_id_to_shuffle_dfs = {}

for file_id, intersected in file_id_to_intersected.items():
    print(file_id)
    
    all_regions = []
    all_indices = []

    for r in intersected.iterrows():
        index = r[0]
        r = r[1]
        chrom = r.loc['chrom_region']
        start = r.loc['start_region']
        end = r.loc['end_region']
        peak_start = r.loc['start']
        peak_end = r.loc['end']
        strand = r.loc['strand_region']

        try:
            n_regions = get_random_regions(chrom, start, end, peak_start, peak_end, strand)
            all_regions.append(n_regions)

            all_indices.append(index)
        except Exception as e:
            print(chrom, start, end, peak_start, peak_end, strand)
            print(e)
            
    shuffle_df = pd.DataFrame(all_regions, index=all_indices)
    shuffle_df = shuffle_df[~shuffle_df.index.duplicated(keep='first')]
    
    print('\t', len(shuffle_df), len(shuffle_df.columns))
    file_id_to_shuffle_dfs[file_id] = shuffle_df

Enzyme_Only_PFA
	 1492 20
RBFOX2_INSCRIBE_PFA
	 2709 20
TDP43_INSCRIBE_PFA
	 2267 20


### From aggregate dfs containing shuffled information, extract each column as a random bed file and save

In [15]:
def expand_string_to_df(label):
    chrom = label.split(':')[0]
    start = label.split(':')[1].split('-')[0]
    end = label.split(':')[1].split('-')[1].split('(')[0]
    strand = label.split('(')[1].split(')')[0]
    return chrom, start, end, strand


file_id_to_list_of_random_instance_dfs = {}


for file_id, shuffle_df in file_id_to_shuffle_dfs.items():
    print(file_id)
    random_instance_dfs = []

    for random_instance_index in shuffle_df.columns:
        print('{}/{}'.format(random_instance_index, len(shuffle_df.columns)))
        random_instance = shuffle_df[[random_instance_index]]
        random_instance_df = pd.DataFrame(zip(*random_instance[random_instance_index].apply(expand_string_to_df))).T
        random_instance_df.columns = ['chrom', 'start', 'end', 'strand']
        random_instance_dfs.append(random_instance_df)

    file_id_to_list_of_random_instance_dfs[file_id] = random_instance_dfs

Enzyme_Only_PFA
0/20
1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20
RBFOX2_INSCRIBE_PFA
0/20
1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20
TDP43_INSCRIBE_PFA
0/20
1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20


### Make directories into which to put shuffles

In [16]:
import subprocess

shuffled_dir = main_dir + '/shuffled_confident_clusters/'

subprocess.run(["mkdir", shuffled_dir])

CompletedProcess(args=['mkdir', '/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/shuffled_confident_clusters/'], returncode=0)

In [17]:
import os

for file_id, list_of_random_instance_dfs in file_id_to_list_of_random_instance_dfs.items():
    print(file_id)
    dir_name = (shuffled_dir + '{}').format(file_id)

    try:
        os.mkdir(dir_name)
    except Exception as e:
        print(e)
        
    # Save beds
    
    for i, df in enumerate(list_of_random_instance_dfs):
        df.to_csv('{}/{}_shuffle{}.tsv'.format(dir_name, file_id, i), index=False, sep='\t')

Enzyme_Only_PFA
RBFOX2_INSCRIBE_PFA
TDP43_INSCRIBE_PFA


### Add sequences and motif presence

In [18]:
shuffled_peak_filepaths = [f for f in glob(shuffled_dir + '*/*') \
                           if 'with_sequence' not in f and 'motif_presence' not in f]
len(shuffled_peak_filepaths)

shuffled_peak_filepaths[0:3]

['/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/shuffled_confident_clusters/TDP43_INSCRIBE_PFA/TDP43_INSCRIBE_PFA_shuffle10.tsv',
 '/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/shuffled_confident_clusters/TDP43_INSCRIBE_PFA/TDP43_INSCRIBE_PFA_shuffle14.tsv',
 '/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/shuffled_confident_clusters/TDP43_INSCRIBE_PFA/TDP43_INSCRIBE_PFA_shuffle4.tsv']

In [19]:
from pyfaidx import Fasta
import re
fasta = '/tscc/lustre/ddn/scratch/q2liang/reference/hg38.fa'
FA = Fasta(fasta, rebuild=False)
import math

def get_sequence(r): 
    chrom = str(r.chrom)
    start = r.start
    end = r.end
    strand = r.strand
    
    sequence = FA[chrom][start:end].seq
    sequence = sequence.upper()
        
    return sequence

def get_extended_sequence(r): 
    chrom = str(r.chrom)
    
    midpoint = r.start + (int((r.end - r.start)/2))
    start = midpoint - 150
    end = midpoint + 150
    strand = r.strand
    
    sequence = FA[chrom][start:end].seq
    sequence = sequence.upper()
        
    return sequence


for peak_filepath in sorted(shuffled_peak_filepaths):
    print('Processing {}...'.format(peak_filepath))
    peak_df = pd.read_csv(peak_filepath, sep='\t')
    filename = peak_filepath.split('/')[-1]
    
    print('\tfilename is {}'.format(filename))
    
    folder = peak_filepath.split(filename)[0]
    print('\tfolder is {}'.format(folder))
    output_filepath = '{}/{}.with_sequence.bed'.format(folder, filename[0:-4])
    print(output_filepath)
    
    peak_df['sequence(+)'] = peak_df.apply(get_sequence, axis=1)
    peak_df['extended_sequence(+)'] = peak_df.apply(get_extended_sequence, axis=1)

    print('\tAssigned sequences')
    peak_df.to_csv(output_filepath, sep='\t', index=False)
    print('\tOutput file at {}'.format(output_filepath))

/tscc/nfs/home/q2liang/miniconda3/envs/py3/lib/python3.6/site-packages/pyfaidx/__init__.py:498: RuntimeWarning: Index file /tscc/lustre/ddn/scratch/q2liang/reference/hg38.fa.fai is older than FASTA file /tscc/lustre/ddn/scratch/q2liang/reference/hg38.fa.
  ), RuntimeWarning


Processing /tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/shuffled_confident_clusters/Enzyme_Only_PFA/Enzyme_Only_PFA_shuffle0.tsv...
	filename is Enzyme_Only_PFA_shuffle0.tsv
	folder is /tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/shuffled_confident_clusters/Enzyme_Only_PFA/
/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/shuffled_confident_clusters/Enzyme_Only_PFA//Enzyme_Only_PFA_shuffle0.with_sequence.bed
	Assigned sequences
	Output file at /tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/shuffled_confident_clusters/Enzyme_Only_PFA//Enzyme_Only_PFA_shuffle0.with_sequence.bed
Processing /tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/shuffled_confident_clusters/Enzyme_Only_PFA/Enzyme_Only_PFA_shuffle1.tsv...
	filename is Enzyme_Only_PFA_shuffle1.tsv
	folder is /tscc/lustre/ddn/scratc